In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 
from sqlalchemy import create_engine, text

PostgreSQL_HOST = '127.0.0.1'
PostgreSQL_PORT = '5432'
PostgreSQL_USER = 'postgres'
PostgreSQL_PASSWORD = '123456'
PostgreSQL_DB = 'sql_advanced'

engine = create_engine('postgresql+psycopg2://%s:%s@%s:%s/%s'
                           % (PostgreSQL_USER, PostgreSQL_PASSWORD, PostgreSQL_HOST, PostgreSQL_PORT, PostgreSQL_DB))

In [2]:
CarSale_df = pd.read_sql_query('SELECT * FROM CarSale', engine) 
CarSale_df

,car_id,sale_dt,sale_count
0,10001,2023-05-01,1
1,10001,2023-05-02,2
2,10001,2023-05-03,3
3,10001,2023-05-04,4
4,10001,2023-06-05,5
5,10001,2023-06-06,6
6,10001,2023-06-07,7
7,10001,2023-06-08,8
8,10002,2023-05-01,10
9,10002,2023-05-02,11


In [3]:
query_1 = """
    select car_id,sale_dt,sale_count, 
        sum(sale_count) over(partition by car_id order by sale_count RANGE BETWEEN CURRENT ROW AND 2 FOLLOWING) as num 
    from CarSale ;
"""
df = pd.read_sql_query(query_1, engine) 
df

,car_id,sale_dt,sale_count,num
0,10001,2023-05-01,1,6
1,10001,2023-05-02,2,9
2,10001,2023-05-03,3,12
3,10001,2023-05-04,4,15
4,10001,2023-06-05,5,18
5,10001,2023-06-06,6,21
6,10001,2023-06-07,7,15
7,10001,2023-06-08,8,8
8,10002,2023-06-05,4,15
9,10002,2023-06-06,5,18


In [4]:
df = pd.read_sql_query("""select * from salarywz""", engine) 
df

,year,person_id,person_name,country_name,salary
0,2021,100,鲁班,战国,10000
1,2022,100,鲁班,战国,10001
2,2023,100,鲁班,战国,9999
3,2022,101,白起,战国,9856
4,2023,101,白起,战国,10003
5,2022,102,蔡文姬,三国,9857
6,2023,102,蔡文姬,三国,10523
7,2020,102,蔡文姬,三国,10006
8,2021,103,孙悟空,唐朝,10007
9,2023,103,孙悟空,唐朝,10325


In [5]:
query_3 = """
select *, 
    MAX(salary) OVER() as max_salay,
    MIN(salary) OVER(partition by country_name) as min_salary_country
from salarywz
"""
df_3 = pd.read_sql_query(query_3, engine) 
df_3

,year,person_id,person_name,country_name,salary,max_salay,min_salary_country
0,2022,102,蔡文姬,三国,9857,10523,9857
1,2020,102,蔡文姬,三国,10006,10523,9857
2,2023,102,蔡文姬,三国,10523,10523,9857
3,2022,103,孙悟空,唐朝,10325,10523,10007
4,2021,103,孙悟空,唐朝,10007,10523,10007
5,2023,103,孙悟空,唐朝,10325,10523,10007
6,2020,103,孙悟空,唐朝,10010,10523,10007
7,2022,100,鲁班,战国,10001,10523,9856
8,2021,100,鲁班,战国,10000,10523,9856
9,2023,101,白起,战国,10003,10523,9856


In [6]:
query_4 = """
select *, 
    sum(salary) OVER() as total_salary,
    sum(salary) OVER(partition by country_name) as salary_country,
    sum(salary) OVER(partition by country_name order by salary ASC) as salary_country_asc_salary,
    sum(salary) OVER(partition by country_name order by salary DESC) as salary_country_desc_salary
from salarywz;
"""
df_4 = pd.read_sql_query(query_4, engine) 
df_4

,year,person_id,person_name,country_name,salary,total_salary,salary_country,salary_country_asc_salary,salary_country_desc_salary
0,2022,102,蔡文姬,三国,9857,120912,30386,9857,30386
1,2020,102,蔡文姬,三国,10006,120912,30386,19863,20529
2,2023,102,蔡文姬,三国,10523,120912,30386,30386,10523
3,2021,103,孙悟空,唐朝,10007,120912,40667,10007,40667
4,2020,103,孙悟空,唐朝,10010,120912,40667,20017,30660
5,2023,103,孙悟空,唐朝,10325,120912,40667,40667,20650
6,2022,103,孙悟空,唐朝,10325,120912,40667,40667,20650
7,2022,101,白起,战国,9856,120912,49859,9856,49859
8,2023,100,鲁班,战国,9999,120912,49859,19855,40003
9,2021,100,鲁班,战国,10000,120912,49859,29855,30004


In [7]:
query_5 = """
select *, 
    row_number() over (partition by person_id order by salary DESC) as odery_row_number,
    rank() over (partition by person_id order by salary DESC) as odery_rank,
    dense_rank() over (partition by person_id order by salary DESC) as odery_dense_rank
from salarywz
"""
df_5 = pd.read_sql_query(query_5, engine) 
df_5

,year,person_id,person_name,country_name,salary,odery_row_number,odery_rank,odery_dense_rank
0,2022,100,鲁班,战国,10001,1,1,1
1,2021,100,鲁班,战国,10000,2,2,2
2,2023,100,鲁班,战国,9999,3,3,3
3,2023,101,白起,战国,10003,1,1,1
4,2022,101,白起,战国,9856,2,2,2
5,2023,102,蔡文姬,三国,10523,1,1,1
6,2020,102,蔡文姬,三国,10006,2,2,2
7,2022,102,蔡文姬,三国,9857,3,3,3
8,2023,103,孙悟空,唐朝,10325,1,1,1
9,2022,103,孙悟空,唐朝,10325,2,1,1


In [8]:
pd.read_sql_query("""select * from salarywz""", engine) 

,year,person_id,person_name,country_name,salary
0,2021,100,鲁班,战国,10000
1,2022,100,鲁班,战国,10001
2,2023,100,鲁班,战国,9999
3,2022,101,白起,战国,9856
4,2023,101,白起,战国,10003
5,2022,102,蔡文姬,三国,9857
6,2023,102,蔡文姬,三国,10523
7,2020,102,蔡文姬,三国,10006
8,2021,103,孙悟空,唐朝,10007
9,2023,103,孙悟空,唐朝,10325


In [10]:
query_6 = """
select *, 
    FIRST_VALUE(salary) over (order by salary DESC) as first_1,
    LAST_VALUE(salary) over (order by salary DESC) as last_1
from salarywz;
"""
pd.read_sql_query(query_6, engine) 

,year,person_id,person_name,country_name,salary,first_1,last_1
0,2023,102,蔡文姬,三国,10523,10523,10523
1,2022,103,孙悟空,唐朝,10325,10523,10325
2,2023,103,孙悟空,唐朝,10325,10523,10325
3,2020,103,孙悟空,唐朝,10010,10523,10010
4,2021,103,孙悟空,唐朝,10007,10523,10007
5,2020,102,蔡文姬,三国,10006,10523,10006
6,2023,101,白起,战国,10003,10523,10003
7,2022,100,鲁班,战国,10001,10523,10001
8,2021,100,鲁班,战国,10000,10523,10000
9,2023,100,鲁班,战国,9999,10523,9999


In [11]:
query_7  = """
select *, 
	FIRST_VALUE(salary) over (partition by country_name order by salary DESC) as first_11,
	LAST_VALUE(salary) over (partition by country_name order by salary DESC) as last_11
from salarywz;

"""
pd.read_sql_query(query_7, engine) 

,year,person_id,person_name,country_name,salary,first_11,last_11
0,2023,102,蔡文姬,三国,10523,10523,10523
1,2020,102,蔡文姬,三国,10006,10523,10006
2,2022,102,蔡文姬,三国,9857,10523,9857
3,2022,103,孙悟空,唐朝,10325,10325,10325
4,2023,103,孙悟空,唐朝,10325,10325,10325
5,2020,103,孙悟空,唐朝,10010,10325,10010
6,2021,103,孙悟空,唐朝,10007,10325,10007
7,2023,101,白起,战国,10003,10003,10003
8,2022,100,鲁班,战国,10001,10003,10001
9,2021,100,鲁班,战国,10000,10003,10000


In [13]:
query_8  = """
select *, 
	FIRST_VALUE(salary) over (partition by country_name order by salary desc RANGE BETWEEN CURRENT ROW AND 1 FOLLOWING ) as first_111,
	LAST_VALUE(salary) over (partition by country_name order by salary desc RANGE BETWEEN CURRENT ROW AND 1 FOLLOWING ) as last_111
from salarywz;

"""
pd.read_sql_query(query_8, engine) 

,year,person_id,person_name,country_name,salary,first_111,last_111
0,2023,102,蔡文姬,三国,10523,10523,10523
1,2020,102,蔡文姬,三国,10006,10006,10006
2,2022,102,蔡文姬,三国,9857,9857,9857
3,2022,103,孙悟空,唐朝,10325,10325,10325
4,2023,103,孙悟空,唐朝,10325,10325,10325
5,2020,103,孙悟空,唐朝,10010,10010,10010
6,2021,103,孙悟空,唐朝,10007,10007,10007
7,2023,101,白起,战国,10003,10003,10003
8,2022,100,鲁班,战国,10001,10001,10000
9,2021,100,鲁班,战国,10000,10000,9999


In [15]:
query_9 = """
select *, 
	NTILE(3) over (order by salary DESC) as ntil_1
from salarywz;

"""
pd.read_sql_query(query_9, engine) 

,year,person_id,person_name,country_name,salary,ntil_1
0,2023,102,蔡文姬,三国,10523,1
1,2022,103,孙悟空,唐朝,10325,1
2,2023,103,孙悟空,唐朝,10325,1
3,2020,103,孙悟空,唐朝,10010,1
4,2021,103,孙悟空,唐朝,10007,2
5,2020,102,蔡文姬,三国,10006,2
6,2023,101,白起,战国,10003,2
7,2022,100,鲁班,战国,10001,2
8,2021,100,鲁班,战国,10000,3
9,2023,100,鲁班,战国,9999,3


In [16]:
query_9 = """
select *, 
	NTILE(3) over (partition by country_name order by salary DESC) as ntil_2
from salarywz;

"""
pd.read_sql_query(query_9, engine) 

,year,person_id,person_name,country_name,salary,ntil_2
0,2023,102,蔡文姬,三国,10523,1
1,2020,102,蔡文姬,三国,10006,2
2,2022,102,蔡文姬,三国,9857,3
3,2022,103,孙悟空,唐朝,10325,1
4,2023,103,孙悟空,唐朝,10325,1
5,2020,103,孙悟空,唐朝,10010,2
6,2021,103,孙悟空,唐朝,10007,3
7,2023,101,白起,战国,10003,1
8,2022,100,鲁班,战国,10001,1
9,2021,100,鲁班,战国,10000,2


In [17]:
query_9 = """
select *, 
	NTILE(3) over (order by salary DESC) as ntil_1,
    NTILE(3) over (partition by country_name order by salary DESC) as ntil_2
from salarywz;

"""
pd.read_sql_query(query_9, engine) 

,year,person_id,person_name,country_name,salary,ntil_1,ntil_2
0,2023,102,蔡文姬,三国,10523,1,1
1,2023,103,孙悟空,唐朝,10325,1,1
2,2022,103,孙悟空,唐朝,10325,1,1
3,2020,103,孙悟空,唐朝,10010,1,2
4,2021,103,孙悟空,唐朝,10007,2,3
5,2020,102,蔡文姬,三国,10006,2,2
6,2023,101,白起,战国,10003,2,1
7,2022,100,鲁班,战国,10001,2,1
8,2021,100,鲁班,战国,10000,3,2
9,2023,100,鲁班,战国,9999,3,2


In [18]:
query_10 = """
select *, 
	LEAD(salary) over (partition by country_name order by salary DESC) as LEAD_RES_1
from salarywz;
"""
pd.read_sql_query(query_10, engine) 

,year,person_id,person_name,country_name,salary,lead_res_1
0,2023,102,蔡文姬,三国,10523,10006.0
1,2020,102,蔡文姬,三国,10006,9857.0
2,2022,102,蔡文姬,三国,9857,NaN
3,2022,103,孙悟空,唐朝,10325,10325.0
4,2023,103,孙悟空,唐朝,10325,10010.0
5,2020,103,孙悟空,唐朝,10010,10007.0
6,2021,103,孙悟空,唐朝,10007,NaN
7,2023,101,白起,战国,10003,10001.0
8,2022,100,鲁班,战国,10001,10000.0
9,2021,100,鲁班,战国,10000,9999.0


In [19]:
query_10 = """
select *, 
	LEAD(salary,2) over (partition by country_name order by salary DESC) as LEAD_RES_1
from salarywz;
"""
pd.read_sql_query(query_10, engine) 

,year,person_id,person_name,country_name,salary,lead_res_1
0,2023,102,蔡文姬,三国,10523,9857.0
1,2020,102,蔡文姬,三国,10006,NaN
2,2022,102,蔡文姬,三国,9857,NaN
3,2022,103,孙悟空,唐朝,10325,10010.0
4,2023,103,孙悟空,唐朝,10325,10007.0
5,2020,103,孙悟空,唐朝,10010,NaN
6,2021,103,孙悟空,唐朝,10007,NaN
7,2023,101,白起,战国,10003,10000.0
8,2022,100,鲁班,战国,10001,9999.0
9,2021,100,鲁班,战国,10000,9856.0


In [20]:
query_10 = """
select *, 
	LAG(salary) over (partition by country_name order by salary DESC) as LEAD_RES_1
from salarywz;
"""
pd.read_sql_query(query_10, engine) 

,year,person_id,person_name,country_name,salary,lead_res_1
0,2023,102,蔡文姬,三国,10523,NaN
1,2020,102,蔡文姬,三国,10006,10523.0
2,2022,102,蔡文姬,三国,9857,10006.0
3,2022,103,孙悟空,唐朝,10325,NaN
4,2023,103,孙悟空,唐朝,10325,10325.0
5,2020,103,孙悟空,唐朝,10010,10325.0
6,2021,103,孙悟空,唐朝,10007,10010.0
7,2023,101,白起,战国,10003,NaN
8,2022,100,鲁班,战国,10001,10003.0
9,2021,100,鲁班,战国,10000,10001.0
